# Ref: https://github.com/sakunaharinda/ragatouille-book/blob/main/book/2_Query_Transformation.ipynb

# Query Transformation

The main idea behind the Query Transformation is that translate/transform the user query in a way that the LLM can correctly answer the question. For instance, if the user asks an ambiguous question, our RAG retriever might retrieve incorrect (or ambiguous) documents based on the embeddings that are not very relevant to answer the user question, leading the LLM to hallucinate answers. There are few ways to tackle this problem. Some of them are,

- [Step-back prompting](https://arxiv.org/pdf/2310.06117): This involves encouraging the LLM to take a step back from a given question or problem and pose a more abstract, higher-level question that encompasses the essence of the original inquiry.
- [Least-to-most prompting](https://arxiv.org/pdf/2205.10625): This allows to break down a complex problem into a series of simpler subproblems and then solve them in sequence. Solving each subproblem is facilitated by the answers to previously solved subproblems.
- Query re-writing ([Multi-Query](https://medium.com/@kbdhunga/advanced-rag-multi-query-retriever-approach-ad8cd0ea0f5b) or [RAG Fusion](https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1)): This allows to generate multiple questions from the original question with different wording and perspectives. Then retrieve documents using the similarity scores between each question and the vector store to answer the orginal question.

A blog post about query transformation by Langchain can be found [here](https://blog.langchain.dev/query-transformations/). 

Now, let's try to implement the above techniques using LangChain!


In [18]:
from dotenv import load_dotenv
load_dotenv()
import rich

Similar to the Introduction notebook, we first import the libraries, load documents, split them, generate embeddings, store them in a vector store and create the retriever using the vector store.

In [19]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.load import loads, dumps
from typing import List

In [20]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# embedding = OpenAIEmbeddings

/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
loader = DirectoryLoader('../../pdf_files/',glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

# Split text into chunks

text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

vectorstore = Chroma.from_documents(documents=text_chunks, 
                                    embedding=embedding,
                                    persist_directory="data/vectorstore")
vectorstore.persist()

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={'k':5})

## Query Translation

### Multi-Query

In multi-query approach, we first use an LLM (here it is an instance of GPT-4) to generate 5 different questions based on our original question. To do that, we create a prompt and encapsulate it with the `ChatPromptTemplate`. Then we create the chain using LCEL, to read the user input and assign it to the `question` placeholder of the prompt, send the prompt to the LLM, parse the output containing 5 questions seperated by new line charcters.

In [23]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    You are an intelligent assistant. Your task is to generate 5 questions based on the provided question in different wording and different perspectives to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newline `\n`.
    
    Original question: {question}
    """
)

generate_queries = (
    {"question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model='gpt-4o-mini', temperature=0.7)
    | StrOutputParser()
    | (lambda x: x.split("\n\n"))
)

We can check whether or not our query generation works by invoking the created chain with a query.

In [24]:
generate_queries.invoke("What need to consider when using LLM to eval LLM generation?")

['What factors should be taken into account when assessing LLM-generated content using another LLM?  \nHow can we effectively evaluate the output of one LLM using another LLM?  \nWhat considerations are important for evaluating LLM outputs with a different LLM model?  \nWhat criteria should guide the evaluation of LLM generations through the lens of another LLM?  \nWhat aspects must be addressed when employing LLMs to critique the generation of other LLMs?  ']

Once we get the 5 questions, we parallelly retrieve the most relevant 5 documents for each question (resulting in a list of lists) and create a new document list by taking the unique documents of the union of all the retrieved documents. To do that we create another chain, `retrieval_chain` using LCEL.

In [25]:
def get_context_union(docs: List[List]):
    all_docs = [dumps(d) for doc in docs for d in doc]
    unique_docs = list(set(all_docs))
    
    return [loads(doc).page_content for doc in unique_docs] # We only return page contents


retrieval_chain = (
    {'question': RunnablePassthrough()}
    | generate_queries
    | retriever.map()
    | get_context_union
)
    

In [26]:
retrieval_chain.invoke("What need to consider when using LLM to eval LLM generation?")

['Model-Based Evaluation Metrics for Text Generation.\nIn Proceedings of the 61st Annual Meeting of the\nAssociation for Computational Linguistics (Volume\n1: Long Papers), 12067–12097. Toronto, Canada:\nAssociation for Computational Linguistics.',
 'ample, The evaluation pipeline of Vicuna (Zheng\net al., 2023) has gained significant interest and wide\nusage due to its simplicity and interpretability. It\nprompts GPT-4 to score and compare candidate\nresponses and provide explanations, making it a\nvaluable tool for evaluation. However, it is un-\nclear how reliable LLMs are as evaluators, as they\nare known to be sensitive to textual instructions\nand inputs (Dong et al., 2022; Turpin et al., 2023;\narXiv:2305.17926v2  [cs.CL]  30 Aug 2023',
 'with human intent (He et al., 2023). While hu-\nman evaluation is treated as the most accurate mea-\nsurement of model performance, it is costly and\ntime-consuming to operate at scales. Consider-\ning the potent capabilities of LLMs, researche

Finally we put all together by creating a one final chain to read the user query, get the contexts from 5 different documents using the `retrieval_chain`, add both the question and context to the prompt, send it through the LLM, and get the final formatted output using  the `StrOutputParser`.

In [27]:
prompt = ChatPromptTemplate.from_template(
    """
    Asnwer the given question using the provided context.\n\nContext: {context}\n\nQuestion: {question}
    """
)

multi_query_chain = (
    {'context': retrieval_chain, 'question': RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model='gpt-4o-mini', temperature=0)
    | StrOutputParser()
)

In [28]:
multi_query_chain.invoke("What need to consider when using LLM to eval LLM generation?")

'When using LLMs to evaluate LLM generation, it is important to consider the following factors:\n\n1. **Reliability of LLMs as Evaluators**: LLMs can be sensitive to textual instructions and inputs, which may affect their evaluation consistency and reliability.\n\n2. **Complexity of Tasks**: Certain complex tasks, such as those requiring common-sense reasoning, may pose challenges for LLMs in accurately assessing performance.\n\n3. **Human Evaluation as a Benchmark**: While human evaluation is considered the most accurate measurement of model performance, it is costly and time-consuming. Therefore, the effectiveness of LLMs as evaluators should be compared against human evaluations.\n\n4. **Interpretability of Evaluation**: The evaluation pipeline should be simple and interpretable, allowing for clear understanding of how LLMs score and compare candidate responses.\n\n5. **Calibration and Robustness**: Calibration techniques may enhance the robustness of LLMs in evaluation tasks, helpi

After executing all the above cells, you will be able to see a LangSmith trace like [this](https://smith.langchain.com/public/31d1e43a-3727-4d0b-82fb-2bbdf146dfac/r).

### RAG Fusion

In the default multi-query approach, after we retrieved the relevant documents for each question generated for our original question, we take the union of all the documents to select only unique documents (same document can be retrieved by multiple questions). However, we did not pay attention to the rank of each document in the context, which is important for the LLM to produce the most correct answer. Because the each individual rank would help us to decide the top-k documents to select as the context if we have a huge number of documents with a limited context window of the LLM. Therefore in RAG Fusion, while we do exactly the same thing upto retrieving documents, we use [Reciprocal Rank Fusion (RRF)](https://learn.microsoft.com/en-us/azure/search/hybrid-search-ranking) to rank the each retrieved document before using them as the context to answer our original question. 

In [29]:
def rrf(results: List[List], k=60):
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0

            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]

            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

            print(f"Updating score for {doc} from {previous_score} to {fused_scores[doc]} based on rank {rank}")

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(ranked_doc), score)
        for ranked_doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

The only difference between the below code compared to the multi-query code we went through earlier is, now we use our `rrf` method instead of `get_context_union` to retrieve the final list of documents related to our original question (i.e., context).

In [30]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    You are an intelligent assistant. Your task is to generate 4 questions based on the provided question in different wording and different perspectives to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newlines. Original question: {question}
    """
)

generate_queries = (
    {"question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model='gpt-4o-mini', temperature=0.7)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


fusion_retrieval_chain = (
    {'question': RunnablePassthrough()}
    | generate_queries
    | retriever.map()
    | rrf
)


In [31]:
result = fusion_retrieval_chain.invoke("What need to consider when using LLM to eval LLM generation?")

TypeError: unhashable type: 'Document'

In [30]:
rich.print(result)

[
    (
        Document(
            metadata={
                'page': 8,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='P.; Tay, Y .; Shazeer, N. M.; Prabhakaran, V .; Reif, E.;\nDu, N.; Hutchinson, B. C.; 
Pope, R.; Bradbury, J.;\nAustin, J.; Isard, M.; Gur-Ari, G.; Yin, P.; Duke, T.;\nLevskaya, A.; Ghemawat, S.; Dev, 
S.; Michalewski,\nH.; García, X.; Misra, V .; Robinson, K.; Fedus, L.;\nZhou, D.; Ippolito, D.; Luan, D.; Lim, H.; 
Zoph, B.;\nSpiridonov, A.; Sepassi, R.; Dohan, D.; Agrawal, S.;\nOmernick, M.; Dai, A. M.; Pillai, T. S.; Pellat, 
M.;\nLewkowycz, A.; Moreira, E.; Child, R.; Polozov, O.;'
        ),
        0.05
    ),
    (
        Document(
            metadata={
                'page': 7,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='the LLMs are already widely adopted as a proxy of\nhuman evaluators, the reliability of 
this paradigm'
        ),
        0.04972677595628415
    ),
    (
        Document(
            metadata={
                'page': 7,
                'source': '../../pdf_files/DiagGPT - An LLM-based Chatbot with Automatic Topic Management for 
Task-Oriented Dialogue(2308.08043).pdf'
            },
            page_content='Yin, P.; Duke, T.; Levskaya, A.; Ghemawat, S.; Dev, S.;\nMichalewski, H.; Garcia, X.; 
Misra, V .; Robinson, K.; Fe-\ndus, L.; Zhou, D.; Ippolito, D.; Luan, D.; Lim, H.; Zoph, B.;\nSpiridonov, A.; 
Sepassi, R.; Dohan, D.; Agrawal, S.; Omer-\nnick, M.; Dai, A. M.; Pillai, T. S.; Pellat, M.; Lewkowycz,\nA.; 
Moreira, E.; Child, R.; Polozov, O.; Lee, K.; Zhou, Z.;\nWang, X.; Saeta, B.; Diaz, M.; Firat, O.; Catasta, M.; 
Wei,\nJ.; Meier-Hellstern, K.; Eck, D.; Dean, J.; Petrov, S.; and'
        ),
        0.04918032786885246
    ),
    (
        Document(
            metadata={
                'page': 1,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='13B in the Vicuna benchmark and demonstrate the\neffectiveness of our proposed approach 
through ex-\nperimental results, which show closer alignment\nwith human judgments.\n2 Positional Bias of the LLM 
Evaluator\n2.1 LLMs as Evaluators\nRecently, researchers have been utilizing LLMs\nsuch as GPT-4 as evaluators to 
compare the per-\nformance of two AI assistants. As shown in Table\n1, an evaluation template with three 
placeholders\nT(Q, R1, R2), is used to query the LLM for eval-'
        ),
        0.048915917503966164
    ),
    (
        Document(
            metadata={
                'page': 7,
                'source': '../../pdf_files/Large Language Models are not Fair Evaluators(2305.17926v2).pdf'
            },
            page_content='with human intent (He et al., 2023). While hu-\nman evaluation is treated as the most 
accurate mea-\nsurement of model performance, it is costly and\ntime-consuming to operate at scales. Consider-\ning
the potent capabilities of LLMs, researchers\nhave started utilizing LLMs to evaluate the profi-\nciency of 
generative models in adhering to human\ninstructions (Zheng et al., 2023; Lu et al., 2023; Li\net al., 2023). In 
these works, Vicuna’s evaluation\nparadigm (Zheng et al., 2023) is widely adopted,'
        ),
        0.04866871479774705
    ),
    (
        Document(
            metadata={
                'page': 6,
                'source': '../../pdf_files/DiagGPT - An LLM-based Chatbot with Automatic Topic Management for 
Task-Oriented Dialogue(2308.08043).pdf'
            },
            page_content="Doctor, I have been experiencing some symptoms recently and I'm concerned that I might 
have contracted COVID-19. Could you please advise me on the necessary tests and precautions I should take?"
        ),
        0.04838709677419355
    ),
    (
        Document(
          

Here we format the context by considering only the page contents without meta data or re-ranking scores. 

In [32]:
def format_context(documents: List):
    return "\n\n".join([doc[0].page_content for doc in documents])


prompt = ChatPromptTemplate.from_template(
    """
    Asnwer the given question using the provided context.\n\nContext: {context}\n\nQuestion: {question}
    """
)

multi_query_chain = (
    {'context': fusion_retrieval_chain | format_context, 'question': RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model='gpt-4o-mini', temperature=0)
    | StrOutputParser()
)

In [33]:
multi_query_chain.invoke("What need to consider when using LLM to eval LLM generation?")

TypeError: unhashable type: 'Document'

After executing all the above cells, you will be able to see a LangSmith trace like [this](https://smith.langchain.com/public/99c5fb68-0ccf-4508-a72d-7c3a7b5e61d2/r).

## Query Decomposition

In "Query Translation", we focused on generating multiple questions from our original question with different perspectives (i.e., translate the query) to improve RAG.  
However, the generated questions all do have the same meaning despite the wording is different, since it is in fact translation. Therefore, the answers for all the questions are somewhat similar. As a result, while the multi-query approach helps avoid ambiguities of the user query by writing it in different ways, `it will not help when the user query is complex` (e.g., a long mathematical computation).

As a solution we can break down (i.e., decompose) the original query into multiple sub-problems (like in recursion or dynamic programming) and answer each sub-problem sequentially/parallelly to derive the answer to our original query. This simplifies the prompts and increases the context for the retrieval process. We do that using `"Query Decomposition"`.

### Least-to-Most Prompting

First let's look at how to implement [Least-to-Most Prompting](https://arxiv.org/pdf/2205.10625) to break down a complex query into subquestions and answer them recursively to derive the final answer. 

Similar to the multi-query and RAG fusion we first have generate a few questions based on our original questions. However our prompt should be different as we are generating sub questions by decomposing the original one, instead of generating the same question with different perspectives. 

In [34]:
from langchain.prompts import ChatPromptTemplate

decompostion_prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful assistant that can break down complex questions into simpler parts. \n
        Your goal is to decompose the given question into multiple sub-questions that can be answerd in isolation to answer the main question in the end. \n
        Provide these sub-questions separated by one newline character. \n
        Original question: {question}\n
        Output (3 queries): 
    """
)

query_generation_chain = (
    {"question": RunnablePassthrough()}
    | decompostion_prompt
    | ChatOpenAI(model='gpt-4o-mini', temperature=0.7)
    | StrOutputParser()
    | (lambda x: x.split("\n\n"))
)


In [35]:
questions = query_generation_chain.invoke("What need to consider when using LLM to eval LLM generation?")
questions

['What are the key metrics to evaluate the quality of LLM-generated content?',
 'What potential biases should be considered when using LLMs for evaluation?',
 'How can human judgment be integrated into the evaluation of LLM outputs?']

After generating the sub-questions, we iterate through them to answer them individually using the `least_to_most_chain`. We first extract the `question` from the user input using the `itemgetter` and provide it to our `retriever` to retrieve related documents as the `context`. `q_a_pairs` will also be provided as part of the user input. Then we populate our prompt and send to the LLM to get the answer. Each time we store the sub-question `Q_{n-1}` and its answer `A_{n-1}` since we provide them as the context to answer the question `Q_{n}`.

In [36]:
from operator import itemgetter


# Create the final prompt template to answer the question with provided context and background Q&A pairs
template = """Here is the question you need to answer:
\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:
\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 
\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

least_to_most_prompt = ChatPromptTemplate.from_template(template) 
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

least_to_most_chain = (
        {'context': itemgetter('question') | retriever,
        'q_a_pairs': itemgetter('q_a_pairs'),
        'question': itemgetter('question'),
        }
        | least_to_most_prompt
        | llm
        | StrOutputParser()
    )

q_a_pairs = ""
for q in questions:
    answer = least_to_most_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})
    q_a_pairs+=f"Question: {q}\n\nAnswer: {answer}\n\n"

After getting answers for the 3 generated sub-questions, finally we answer our original question by invoking the `least_to_most_chain` once again, but this time with the original question and all `q_a_pairs`.

In [37]:
result = least_to_most_chain.invoke({"question": "What need to consider when using LLM to eval LLM generation?", "q_a_pairs": q_a_pairs})

In [39]:
rich.print(result)

When using Large Language Models (LLMs) to evaluate the generation of other LLMs, several important considerations 
should be taken into account:

1. **Reliability of LLMs as Evaluators**: LLMs may not always provide reliable evaluations due to their sensitivity
to textual instructions and inputs. Variations in phrasing or context can lead to inconsistent assessments, which 
raises questions about their effectiveness as evaluators.

2. **Positional Bias**: The order in which responses are presented can significantly influence evaluation outcomes.
LLMs may exhibit positional bias, where the evaluation of a response can vary depending on whether it is assessed 
first or last. This can skew results and affect the perceived quality of the generated content.

3. **Instruction Sensitivity**: LLMs can be highly sensitive to the specific wording and structure of prompts. This
means that slight changes in how a task is framed can lead to different evaluation results, making it crucial to 
standardize evaluation prompts to ensure consistency.

4. **Bias and Fairness**: LLMs may inadvertently perpetuate biases present in their training data, which can affect
their evaluations. It is essential to analyze the outputs for potential biases and ensure that the evaluations do 
not reinforce stereotypes or unfair representations.

5. **Calibration of Evaluators**: The biases of human evaluators can influence how they interpret LLM evaluations. 
Calibrating human evaluators to recognize and minimize their biases is important for achieving fair assessments and
aligning evaluations with objective quality metrics.

6. **Contextual Bias**: The context in which the evaluation occurs can introduce bias. LLMs trained on specific 
cultural or societal norms may evaluate content through a limited lens, potentially overlooking alternative 
perspectives. It is important to consider the diversity of contexts when interpreting evaluations.

7. **Combining Human Judgment with LLM Evaluations**: Integrating human judgment into the evaluation process can 
enhance the reliability and depth of assessments. Human evaluators can provide insights that LLMs may miss, 
particularly regarding nuances and contextual factors.

8. **Iterative Evaluation Process**: Employing an iterative evaluation process where LLM outputs are continuously 
assessed and refined based on feedback can help identify recurring issues and improve the model's performance over 
time.

9. **Task-Specific Guidelines**: Developing specific evaluation criteria based on the type of content being 
assessed can enhance the relevance and accuracy of evaluations. Different tasks may require different metrics for 
quality assessment.

By considering these factors, users can better navigate the complexities of using LLMs for evaluating LLM-generated
content, leading to more reliable and fair assessments.

The LangSmith trace for the original question answer will look like [this](https://smith.langchain.com/public/7bd7f987-a53a-4d32-abb0-823940bc3f27/r).

Instead sequentially answering the sub-questions, we can use the LLM to answer them parallely and use those answers to derive the final answer to our main question.

In [41]:
prompt = hub.pull('rlm/rag-prompt')
rich.print(prompt)

ChatPromptTemplate(
    input_variables=['context', 'question'],
    input_types={},
    partial_variables={},
    metadata={
        'lc_hub_owner': 'rlm',
        'lc_hub_repo': 'rag-prompt',
        'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'
    },
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'],
                input_types={},
                partial_variables={},
                template="You are an assistant for question-answering tasks. Use the following pieces of retrieved 
context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences 
maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
            ),
            additional_kwargs={}
        )
    ]
)

In [43]:
def generate_and_answer(question):
    
    questions = []
    
    sub_questions = query_generation_chain.invoke(question)
    
    sub_qa_chain = (
        {'context': RunnablePassthrough() | retriever, 'question': RunnablePassthrough()}
        | prompt
        | ChatOpenAI(model='gpt-4o-mini', temperature=0)
        | StrOutputParser()
    )
    
    for q in sub_questions:
        answer = sub_qa_chain.invoke(q)
        questions.append({"question": q, "answer": answer})
        
    return questions

In [44]:
result = qa_pairs = generate_and_answer("What need to consider when using LLM to eval LLM generation?")

In [45]:
rich.print(result)

[
    {
        'question': 'What specific metrics or criteria should be used to evaluate LLM generation?',
        'answer': 'To evaluate LLM generation, metrics such as adherence to human instructions and the ability to 
provide coherent and contextually relevant responses are essential. The Vicuna evaluation paradigm is noted for its
simplicity and interpretability, prompting models like GPT-4 to score and compare responses. Additionally, a 
fine-grained analysis of evaluation quality can be performed by categorizing questions and assessing performance 
across these categories.'
    },
    {
        'question': 'What are the potential biases or limitations of the LLM that may affect its generation?',
        'answer': 'The potential biases or limitations of LLMs include sensitivity to the position of responses, 
which can lead to different evaluation results based on the order in which options are presented. Additionally, 
LLMs may be influenced by textual instructions and inputs, raising concerns about their reliability as evaluators. 
These factors can significantly affect the consistency and fairness of their generated outputs.'
    },
    {
        'question': 'How can the context or input provided to the LLM influence the quality of its generation?',
        'answer': 'The context or input provided to the LLM can significantly influence the quality of its 
generation, as LLMs are sensitive to textual instructions and inputs. This sensitivity means that variations in the
prompt can lead to different interpretations and outputs, affecting the overall performance. Consequently, the 
clarity and specificity of the input are crucial for achieving desired results.'
    }
]

In [49]:
def format_qa_pairs(qa_pairs):
    
    formatted_string = ""
    
    for i, qa in enumerate(qa_pairs):
        formatted_string += f"Question {i}: {qa['question']}\nAnswer {i}: {qa['answer']}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(qa_pairs)

# Prompt

prompt = ChatPromptTemplate.from_template(
    """
    Consider the following Question and Answer Pairs:

    {context}

    Use these to synthesize an answer to the question: {question}
    """
)

final_rag_chain = (
     prompt
    | ChatOpenAI(model='gpt-4o-mini', temperature=0)
    | StrOutputParser()
)

In [50]:
result = final_rag_chain.invoke({'context': context, 'question': "What need to consider when using LLM to eval LLM generation?"})

In [51]:
rich.print(result)

When using LLMs to evaluate LLM generation, several key factors need to be considered:

1. **Evaluation Metrics**: It's important to establish specific metrics or criteria for evaluation, such as 
adherence to human instructions, coherence, and contextual relevance of responses. Utilizing frameworks like the 
Vicuna evaluation paradigm can enhance the interpretability and simplicity of the evaluation process.

2. **Biases and Limitations**: Be aware of potential biases inherent in LLMs, such as sensitivity to the order of 
responses and the influence of textual instructions. These biases can lead to inconsistent evaluation results and 
may compromise the fairness and reliability of the outputs.

3. **Context Sensitivity**: The quality of the LLM's generation is heavily influenced by the context or input 
provided. Variations in prompts can lead to different interpretations and outputs, so it is crucial to ensure that 
inputs are clear and specific to achieve the desired evaluation outcomes.

4. **Fine-Grained Analysis**: Conducting a fine-grained analysis by categorizing questions and assessing 
performance across these categories can provide deeper insights into the evaluation quality and help identify 
specific areas for improvement.

By considering these factors, one can enhance the effectiveness and reliability of using LLMs for evaluating LLM 
generation.

The LangSmith trace for answering the original question will look like [this](https://smith.langchain.com/public/d5a17200-7752-42cb-87b9-146959e691bc/r).

### Step back prompting

[Step back prompting](https://arxiv.org/pdf/2310.06117) allows LLMs to step back through in-context learning – prompting them to derive high-level abstractions such as concepts and principles for a specific example (i.e., Abstraction). Then, grounded on the documents regarding the high-level concept or principle, the LLM can reason about the solution to the original question (i.e., Reasoning).

E.g., If the original question is "What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor of 2 and the volume is increased by a factor of 8?", a possible step-back question would be "What are the physics principles behind this question?". Then the context (i.e., documents) retrieved for the step-back question will be used as additional context to answer the original question.

To generate such step-back questions, we use few-shot learning to provide a few examples of (question, step-back question) pairs to the LLM.

In [52]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        'input': 'What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor of 2 and the volume is increased by a factor of 8?',
        'output': 'What are the physics principles behind this question?'
    },
    {
        'input': 'Estella Leopold went to which school between Aug 1954 and Nov 1954?',
        'output': "What was Estella Leopold's education history?"
    }
]
example_prompt = ChatPromptTemplate.from_messages(
            [
                ('human', '{input}'), ('ai', '{output}')
            ]
        )
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
            # This is a prompt template used to format each individual example.
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
            [
                ('system', """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""),
                few_shot_prompt,
                ('user', '{question}'),
            ]
        )

"System: You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:\nHuman: What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor of 2 and the volume is increased by a factor of 8?\nAI: What are the physics principles behind this question?\nHuman: Estella Leopold went to which school between Aug 1954 and Nov 1954?\nAI: What was Estella Leopold's education history?\nHuman: What need to consider when using LLM to eval LLM generation?"

In [54]:
rich.print(final_prompt.format(question= "What need to consider when using LLM to eval LLM generation?"))

System: You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic
step-back question, which is easier to answer. Here are a few examples:
Human: What happens to the pressure, P, of an ideal gas if the temperature is increased by a factor of 2 and the 
volume is increased by a factor of 8?
AI: What are the physics principles behind this question?
Human: Estella Leopold went to which school between Aug 1954 and Nov 1954?
AI: What was Estella Leopold's education history?
Human: What need to consider when using LLM to eval LLM generation?

Then we use the created few-shot prompt to generate the step-back question through a chain.

#### chain type 1

In [62]:
step_back_query_chain = (
    {'question': RunnablePassthrough()}
    | final_prompt 
    | ChatOpenAI(model='gpt-4o-mini', temperature=0.7) 
    | StrOutputParser()
    )

step_back_query_chain.invoke("What need to consider when using LLM to eval LLM generation?")



'What factors should be taken into account when evaluating the output of language models?'

#### chain type 2

In [61]:
step_back_query_chain = (
    final_prompt 
    | ChatOpenAI(model='gpt-4o-mini', temperature=0.9) 
    | StrOutputParser()
    )

step_back_query_chain.invoke({"question": "What need to consider when using LLM to eval LLM generation?"})



'What factors should be taken into account when evaluating the output of language models?'

Finally, we use both the context retrieved for the original question and the context retrieved for the step-back question to answer our original question via the `step_back_chain`.  

In [64]:
response_prompt_template = """You are an expert of world knowledge. 
I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. 
Otherwise, ignore them if they are not relevant.

<normal_context>
# {normal_context}
</normal_context>

<step_back_context>
# {step_back_context}
</step_back_context>


# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

step_back_chain = (
    {'normal_context': RunnablePassthrough() | retriever,
     'step_back_context': RunnablePassthrough() | step_back_query_chain | retriever,
     'question': RunnablePassthrough()
     }
    | response_prompt
    | ChatOpenAI(model='gpt-4o-mini', temperature=0)
    | StrOutputParser()
)

In [58]:
step_back_chain.invoke("What need to consider when using LLM to eval LLM generation?")

'When using large language models (LLMs) to evaluate the generation of other LLMs, several important considerations should be taken into account:\n\n1. **Reliability of LLMs as Evaluators**: LLMs are increasingly being used as proxies for human evaluators due to their efficiency and scalability. However, their reliability in this role is still under scrutiny. Research indicates that LLMs can be sensitive to the textual instructions and inputs they receive, which may lead to inconsistent evaluations (Dong et al., 2022; Turpin et al., 2023). This sensitivity can affect the fairness and accuracy of the evaluation process.\n\n2. **Evaluation Paradigms**: The evaluation pipeline, such as that used by Vicuna, prompts LLMs like GPT-4 to score and compare candidate responses while providing explanations. While this method is valued for its simplicity and interpretability, it is essential to critically assess how well these evaluations align with human judgment and the specific criteria being m

In [66]:
test_step_back_chain = (
    {'normal_context': RunnablePassthrough(),
     'step_back_context': RunnablePassthrough() | step_back_query_chain,
     'question': RunnablePassthrough()
     }
    | response_prompt)

res = test_step_back_chain.invoke("What need to consider when using LLM to eval LLM generation?")
rich.print(res)

ChatPromptValue(
    messages=[
        HumanMessage(
            content='You are an expert of world knowledge. \nI am going to ask you a question. Your response should
be comprehensive and not contradicted with the following context if they are relevant. \nOtherwise, ignore them if 
they are not relevant.\n\n<normal_context>\n# What need to consider when using LLM to eval LLM 
generation?\n</normal_context>\n\n<step_back_context>\n# What factors should be taken into account when assessing 
the output of language models?\n</step_back_context>\n\n\n# Original Question: What need to consider when using LLM
to eval LLM generation?\n# Answer:',
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

The LangSmith trace for the implemented step-back prompting chain will look like [this](https://smith.langchain.com/public/425c098b-47ae-4f53-9259-8cd6b567a2b0/r).

In this notebook, we looked at ways to improve the LLMs answers to a user query through the "Query Transformation". In summary, query transformation may help us to remove ambiguities of the user query and simplify it through techniques such as ,

- **Multi-query**: That re-writes the question in different perspectives (i.e., sub-questions).
- **RAG Fusion**: That not only re-writes the question in different perspectives, but also rank the documents retrieved for each sub-question to provide the most relevant information to answer the original question.

- **Least-to-Most Prompting**: That helps break-down complex questions into mutiple sub problems and answer the final question using the sub problems and their answers as the context.
- **Step-back Prompting**: That generates a step-back question and use the retrieved documents for that step-back question as the additional context to answer the original question.

In the next section, we will  generate Hypothetical Documents, instead of questions to help LLMs answer questions more accurately through [HyDE](https://arxiv.org/pdf/2212.10496) (Hypothetical Document Embeddings).